In [1]:
from keras.models import Model
from keras.layers import Input, Activation, concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Conv2D, MaxPool2D
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import to_categorical
from keras import backend as K

In [3]:
import pickle

# Model
## SqueezeNet

In [4]:
# Refer from : https://github.com/rcmalli/keras-squeezenet/blob/master/keras_squeezenet/squeezenet.py

sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    x = Conv2D(squeeze, (1,1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)
    
    left = Conv2D(expand, (1,1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)
    
    right = Conv2D(expand, (3,3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)
    
    #x = add([left, right], name=s_id + 'concat')
    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    
    return x
    

In [5]:
# Refer from : https://github.com/DT42/squeezenet_demo/blob/master/model.py

def squeezeNet(nb_classes, input_shape=(32,32,3)):
    inputs = Input((input_shape))
    x = Conv2D(96, (7,7), activation='relu', strides=(2,2), padding='same', name='conv1')(inputs)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool2')(x)
    
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = MaxPool2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)
    
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)
    
    x = Conv2D(nb_classes, (1,1), activation='relu', padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    
    softmax = Activation("softmax", name='softmax')(x)
    
    model = Model(inputs=inputs, outputs=softmax)
    model.summary()
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
           
    return model
    

In [6]:
model = squeezeNet(nb_classes=43)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 16, 16, 96)    14208       input_1[0][0]                    
____________________________________________________________________________________________________
maxpool1 (MaxPooling2D)          (None, 7, 7, 96)      0           conv1[0][0]                      
____________________________________________________________________________________________________
fire2/squeeze1x1 (Conv2D)        (None, 7, 7, 16)      1552        maxpool1[0][0]                   
___________________________________________________________________________________________

In [15]:
model_checkpoint = ModelCheckpoint('weights/model_squeezeNet_TSR.hdf5', monitor='loss', save_best_only=True)
model_earlystopping = EarlyStopping(monitor='loss',patience=5)

# Train and Test Model

In [8]:
training_path = "./dataset/Final_Training/"
test_path = "./dataset/Final_Test/"

train_file = training_path + "train.p"
test_file = test_path + "test.p"

num_classes = 43
batch_size = 256
epochs = 1000

with open(train_file, mode='rb') as f:
    X_train, y_train = pickle.load(f)
with open(test_file, mode='rb') as f:
    X_test, y_test = pickle.load(f)

In [9]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [10]:
## Data Augmentation using Keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_datagen.fit(X_train)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train)/batch_size, 
                    epochs=epochs, 
                    callbacks=[model_checkpoint, model_earlystopping])

Epoch 1/1000
154/153 [==============================] - 16s - loss: 0.3785 - acc: 0.8789    
Epoch 2/1000
154/153 [==============================] - 15s - loss: 0.3445 - acc: 0.8919    
Epoch 3/1000
154/153 [==============================] - 15s - loss: 0.3085 - acc: 0.9033    
Epoch 4/1000
154/153 [==============================] - 15s - loss: 0.3064 - acc: 0.9040    
Epoch 5/1000
154/153 [==============================] - 16s - loss: 0.2572 - acc: 0.9205    
Epoch 6/1000
154/153 [==============================] - 15s - loss: 0.2420 - acc: 0.9263    
Epoch 7/1000
154/153 [==============================] - 16s - loss: 0.2254 - acc: 0.9316    
Epoch 8/1000
154/153 [==============================] - 15s - loss: 0.2229 - acc: 0.9332    
Epoch 9/1000
154/153 [==============================] - 15s - loss: 0.1940 - acc: 0.9424    
Epoch 10/1000
154/153 [==============================] - 16s - loss: 0.1893 - acc: 0.9430    
Epoch 11/1000
154/153 [==============================] - 17s - loss: 

In [24]:
scores = model.evaluate_generator(test_datagen.flow(X_test, y_test, batch_size=batch_size*2), steps=len(X_test)/batch_size)
#print("Accuracy = {:0.2f}".format(scores[1]))
print("Accuracy = {:0.4f}".format(scores[1]))

Accuracy = 0.7552
